In [1]:
# Develop features for demo, etc. 
# But, first step is to create DF from Recommendation
import collections
import compress_pickle
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")
RESULT_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS_suppl/result_files'

from AMAS import annotation_maker as am
from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [2]:
E_COLI_PATH = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS/AMAS/../tests/e_coli_core.xml'
BIOMD_190_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000190.xml')

In [3]:
recom = recommender.Recommender(libsbml_fpath=BIOMD_190_PATH)

In [4]:
recom.getSpeciesIDs()

['SAM', 'A', 'P', 'S', 'D', 'aS', 'aD', 'Met', 'ORN', 'AcCoA', 'CoA']

In [5]:
recom.recommendSpecies(ids='SAM', min_score=0.8)

                      SAM (credibility score: 0.975)                      
+----+--------------+---------------+------------------------------------+
|    | annotation   |   match score | label                              |
+====+==============+===============+====================================+
|  1 | CHEBI:59789  |         1.000 | S-adenosyl-L-methionine zwitterion |
+----+--------------+---------------+------------------------------------+
|  2 | CHEBI:15414  |         1.000 | S-adenosyl-L-methionine            |
+----+--------------+---------------+------------------------------------+



In [56]:
glucosc_df = recom.getSpeciesRecommendation(pred_str='glucosc', get_df=True)
glucosc_df

,annotation,match score,label
glucosc (cred. 0.943),,,
1,CHEBI:42758,0.882,aldehydo-D-glucose
2,CHEBI:4167,0.882,D-glucopyranose
3,CHEBI:17234,0.882,glucose


In [60]:
glucosc_df['match score'] = np.round(1/(1.118), 3)

In [61]:
glucosc_df

,annotation,match score,label
glucosc (cred. 0.943),,,
1,CHEBI:42758,0.894,aldehydo-D-glucose
2,CHEBI:4167,0.894,D-glucopyranose
3,CHEBI:17234,0.894,glucose


In [64]:
print(recom.getMarkdownFromRecommendation(glucosc_df))

            glucosc (credibility score: 0.943)            
+----+--------------+---------------+--------------------+
|    | annotation   |   match score | label              |
+====+==============+===============+====================+
|  1 | CHEBI:42758  |         0.894 | aldehydo-D-glucose |
+----+--------------+---------------+--------------------+
|  2 | CHEBI:4167   |         0.894 | D-glucopyranose    |
+----+--------------+---------------+--------------------+
|  3 | CHEBI:17234  |         0.894 | glucose            |
+----+--------------+---------------+--------------------+


In [44]:
df_name = a_df.index.name
a_df = a_df.loc[[4, 5, 10], :]
a_df.index=[1,2,3]
a_df.index.name = df_name
a_df

,annotation,match score,label
a (cred. 0.409),,,
1,CHEBI:16708,1.0,adenine
2,CHEBI:16449,1.0,alanine
3,CHEBI:13193,1.0,hydrogen acceptor


In [45]:
a_df.loc[1, 'match score'] = 1/0.5

In [46]:
print(recom.getMarkdownFromRecommendation(a_df))

               a (credibility score: 0.409)              
+----+--------------+---------------+-------------------+
|    | annotation   |   match score | label             |
+====+==============+===============+===================+
|  1 | CHEBI:16708  |         2.000 | adenine           |
+----+--------------+---------------+-------------------+
|  2 | CHEBI:16449  |         1.000 | alanine           |
+----+--------------+---------------+-------------------+
|  3 | CHEBI:13193  |         1.000 | hydrogen acceptor |
+----+--------------+---------------+-------------------+


In [6]:
recom.selectAnnotation(choice=('SAM', [2]))

Selection updated.


In [7]:
recom.getSpeciesIDs(pattern="S*S")

In [8]:
print(recom.getReactionIDs(pattern="S*S"))

['SSAT_for_S', 'SpdS', 'SpmS']


In [10]:
recom.recommendReaction(ids=['ODC', 'SAMdc'], min_score=0.4)

                                        ODC (credibility score: 0.817)                                       
+----+--------------+---------------+-----------------------------------------------------------------------+
|    | annotation   |   match score | label                                                                 |
+====+==============+===============+=======================================================================+
|  1 | RHEA:28827   |         1.000 | L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out) |
+----+--------------+---------------+-----------------------------------------------------------------------+
|  2 | RHEA:22964   |         0.500 | ornithine decarboxylase activity                                      |
+----+--------------+---------------+-----------------------------------------------------------------------+
|  3 | RHEA:59048   |         0.500 | D-ornithine + H(+) = CO2 + putrescine                                 |
+----+----

In [11]:
recom.selectAnnotation(choice=('ODC', [1,2]))

Selection updated.


In [12]:
recom.displaySelection()

                      SAM (credibility score: 0.975)                      
+----+--------------+---------------+------------------------------------+
|    | annotation   |   match score | label                              |
+====+==============+===============+====================================+
|  2 | CHEBI:59789  |         1.000 | S-adenosyl-L-methionine zwitterion |
+----+--------------+---------------+------------------------------------+

                                        ODC (credibility score: 0.817)                                       
+----+--------------+---------------+-----------------------------------------------------------------------+
|    | annotation   |   match score | label                                                                 |
+====+==============+===============+=======================================================================+
|  1 | RHEA:28827   |         1.000 | L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out) |

In [9]:
recom.saveToFile(os.path.join(os.getcwd(), 'updated_model.xml'))

Annotation of SAM updated.


In [29]:
recom.getSpeciesRecommendation(pred_str='a',get_df=True)

,annotation,match score,label
a (cred. 0.409),,,
1,CHEBI:13193,1.0,hydrogen acceptor
2,CHEBI:28515,1.0,acetoacetamide
3,CHEBI:62885,1.0,adenosine residue
4,CHEBI:15843,1.0,arachidonic acid
5,CHEBI:46217,1.0,L-alanine residue
6,CHEBI:50306,1.0,adenosine 5'-monophosphate residue
7,CHEBI:16449,1.0,alanine
8,CHEBI:73313,1.0,Ala-Ala-Ala
9,CHEBI:72816,1.0,L-alanyl-L-alanine


In [17]:
for val in recommender.ELEMENT_TYPES:
  print(list(recom.selection[val].keys()))

['SAM']
['ODC']


In [36]:
# finally, check if the file has been successfully updated
reader = libsbml.SBMLReader()
document = reader.readSBML(os.path.join(os.getcwd(), 'updated_model.xml'))
model = document.getModel()

In [39]:
print(model.getSpecies('SAM').getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vcard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>
          <bqbiol:hasProperty>
            <rdf:Bag>
              <rdf:li rdf:resource="http://reproduciblebiomodels.org/amas/v1/by_name/1.00"/>
            </rdf:Bag>
          </bqbiol:hasProperty>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [26]:
# Last step is to save them as annotation;
choice = ['SAM']
model = recom.sbml_document.getModel()
ELEMENT_FUNC = {'species': model.getSpecies,
                'reaction': model.getReaction}
for one_type in recommender.ELEMENT_TYPES:
  type_selection = recom.selection[one_type]
  maker = am.AnnotationMaker(one_type)
  if choice: 
    sel2save = {val:type_selection[val] for val in choice if val in type_selection.keys()}
  else:
    sel2save = type_selection
  for one_k in sel2save.keys():
    one_element = ELEMENT_FUNC[one_type](one_k)
    meta_id = one_element.meta_id
    df = sel2save[one_k]
    upd_tuples = list(zip(df['annotation'], df['match score']))
    annotation_str = maker.getAnnotationString(upd_tuples, meta_id)
    one_element.setAnnotation(annotation_str)
    print("elemen %s, annotation updated." % one_k)

elemen SAM, annotation updated.


In [27]:
sel2save

{}

In [17]:


for one_k in sel2save.keys():
  one_element = ELEMENT_FUNC[one_type](one_k)
  meta_id = one_element.meta_id
  df = sel2save[one_k]
  upd_tuples = list(zip(df['annotation'], df['match score']))
  annotation_str = maker.getAnnotationString(upd_tuples, meta_id)
  one_element.setAnnotation(annotation_str)

In [18]:
print(one_element.getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vcard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:59789"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:59789"/>
          <bqbiol:hasProperty>
            <rdf:Bag>
              <rdf:li rdf:resource="http://reproduciblebiomodels.org/amas/v1/by_name/1.00"/>
            </rdf:Bag>
          </bqbiol:hasProperty>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [19]:
ONE_CHEBI = 'CHEBI:15414'
ONE_CHEBI_SCORE = 1.00
TWO_CHEBI = 'CHEBI:59789'
TWO_CHEBI_SCORE = 1.00
CANDIDATES = [(ONE_CHEBI, ONE_CHEBI_SCORE),
              (TWO_CHEBI, TWO_CHEBI_SCORE)]

In [21]:
print(maker.getAnnotationString(CANDIDATES, meta_id='metaid_00001'))

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vcard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>
          <bqbiol:hasProperty>
            <rdf:Bag>
              <rdf:li rdf:resource="http://reproduciblebiomodels.org/amas/v1/by_name/1.00"/>
            </rdf:Bag>
          </bqbiol:hasProperty>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:59789"/>
          <bqbiol:hasProperty>
            <rdf:Bag>
              <rdf:li rdf:resource="http://reproduciblebiomodels.org/amas/v1/by_name/1.00"/>
            </rdf:Bag>
          </bqbiol:hasProperty>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</an

In [131]:
ELEMENT_TYPES = ['speies', 'reaction']
current_type = None
just_displayed = None
selection = {val:dict() for val in ELEMENT_TYPES}

# def filterDataFrameByThreshold(df, min_score):
#   """
#   Filter dataframe by min_score (threshold),
#   and returns the result;
  
#   Note that if no item meets the threshold,
#   it'll still return an empty dataframe. 

#   Paramters
#   ---------
#   df: pd.DataFrame
  
#   min_score: float (0.0-1.0)
  
#   Returns
#   -------
#   pd.DataFrame  
#   """
#   scores = df['match score']
#   filt_idx = scores[scores>=min_score].index
#   filt_df = df.loc[filt_idx, :]
#   return filt_df
 
# def recommendSpecies(ids, min_score=0.0):
#   """
#   Recommend one or more ids of species
#   and returns a single dataframe or
#   a list of dataframes.
  
#   Parameters
#   ----------
#   ids: str/list-str
  
#   min_score: threshold for cutoff
#       If None given, returns all values; 

#   Returns
#   -------
#   None
#   """
#   updateCurrentElementType('species')
  
#   if isinstance(ids, str):
#     species_list = [ids]
#   else:
#     species_list = ids
#   res = recom.getSpeciesListRecommendation(pred_ids=species_list,
#                                            get_df=True,
#                                            threshold=min_score)
#   res_dict = {val:res[idx] for idx, val in enumerate(species_list)}
#   for k in res_dict.keys():
#     filt_df = filterDataFrameByThreshold(res_dict[k], min_score)
#     print(recom.getMarkdownFromRecommendation(filt_df)+"\n")
#     # print("\n")
#   updateJustDisplayed(res_dict)
#   return res_dict

# def recommendReaction(ids, min_score=0.0):
#   """
#   Recommend one or more ids of species
#   and returns a single dataframe or
#   a list of dataframes.
  
#   Parameters
#   ----------
#   ids: str/list-str
  
#   min_score: threshold for cutoff
#       If None given, returns all values; 

#   Returns
#   -------
#   None
#   """
#   updateCurrentElementType('reaction')
#   if isinstance(ids, str):
#     reaction_list = [ids]
#   else:
#     reaction_list = ids
#   res = recom.getReactionListRecommendation(pred_ids=reaction_list,
#                                             get_df=True)
#   res_dict = {val:res[idx] for idx, val in enumerate(reaction_list)}
#   for k in res_dict.keys():
#     filt_df = filterDataFrameByThreshold(res_dict[k], min_score)
#     print(recom.getMarkdownFromRecommendation(filt_df)+"\n")
#     # print("\n")
#   updateJustDisplayed(res)
#   return res_dict

# def selectAnnotation(choice=None):
#   """
#   Based on the previous recommendation,
#   determine the annotations to store.
#   If 'all' given in choice[1],
#   select all.
  
#   Parameters
#   ----------
#   choice: tuple/list-tuple (str, int)
#       [(element ID, choice number)]
  
#   df: pandas.DataFrame
#   """
#   # assumes self.just_displayced is {id: pd.dataframe}
#   sel_id = choice[0]
#   sel_idx = choice[1]
#   df = just_displayed[choice[0]]
#   if sel_idx == 'all':
#     result = df
#   else:
#     if isinstance(sel_idx, int):
#       chosen = [sel_idx]
#     else:
#       chosen = sel_idx
#     result = df.loc[chosen, :]
#   # Now, update the selected annotation
#   updateSelection(sel_id, result)
#   print("Selection updated.")
#   return {sel_id: result}

# def updateCurrentElementType(element_type):
#   """
#   Updating self.current_type
#   indicator; updated when
#   recommendSpecies or recommendReaction 
#   is called; 
  
#   Parameters
#   ----------
#   element_type: str
#       Either 'species' or 'reaction'
#   """
#   current_type = element_type

# def updateJustDisplayed(df_dict):
#   """
#   Used it every time
#   result is shown to user.
#   called by 
#   /recommendSpecies/recommendReaction/
#   /selectAnnotation/
#   For now, always in the format as
#   pandas.DataFrame. 

#   Parameters
#   ----------
#   df_dict: dict()
#       Dictionary of pandas.DataFrame
  
#   Returns
#   -------
#   None
#   """
#   just_displayed = df_dict

# def updateSelection(sel_id, sel_df):
#   """
#   Direct result of selectAnnotation;
#   filtered or non-filtered
#   dictionary of dataframes.
  
#   By calling SaveFile,
#   All selected annotations will be
#   saved as an .xml file. 
  
#   Parameters
#   ----------
#   sel_id: str
  
#   sel_df: pandas.DataFrame
#   """
#   selection[current_type].update({sel_id: sel_df})

    
# def displaySelection():
#   """
#   To assist user, 
#   display all selected
#   annotations from
#   self.selection.
#   """
#   for one_type in ELEMENT_TYPES:
#     type_selection = selection[one_type]
#     for k in type_selection.keys():
#       print(recom.getMarkdownFromRecommendation(type_selection[k])+"\n")
    
    
# def SaveFile(fpath="updated_model.xml", choice=None):
#   """
#   Update and save model;
#   How to distinguish species vs. reactions? 
#   by using self.current_element_type
  
#   Call annotation maker;
  
#   Parameters
#   ----------
#   filepath: str
  
#   choice: list-str/None
#       If list of ID given, 
#       only annotations of the provided IDs 
#       will be saved.
#       If None, all will be saved;
#   """
#   model = recom.sbml_document.getModel()
#   ELEMENT_FUNC = {'species': model.getSpecies,
#                  'reaction': model.getReaction}
#   for one_type in recommender.ELEMENT_TYPES:
#     type_selection = recom.selection[one_type]
#     maker = am.AnnotationMaker(one_type)
#     if choice: 
#       sel2save = {val:type_selection[val] for val in choice if val in type_selection.keys()}
#     else:
#       sel2save = type_selection
#     for one_k in sel2save.keys():
#       one_element = ELEMENT_FUNC[one_type](one_k)
#       meta_id = one_element.meta_id
#       df = sel2save[one_k]
#       upd_tuples = list(zip(df['annotation'], df['match score']))
#       annotation_str = maker.getAnnotationString(upd_tuples, meta_id)
#       one_element.setAnnotation(annotation_str)
#       print("elemen %s, annotation updated." % one_k)  
#   # finally, write the sbml document 
#   libsbml.writeSBMLToFile(recom.sbml_document, fpath)


In [35]:
doc = reader.readSBML("updated_model.xml")
m = doc.getModel()
print(m.getSpecies(one_s.getId()).getAnnotationString())

NameError: name 'reader' is not defined

In [32]:
print(recom.getSpeciesRecommendation(pred_str='a', get_df=True))

                  annotation  match score                               label
a (cred. 0.409)                                                              
1                CHEBI:13193          1.0                   hydrogen acceptor
2                CHEBI:28515          1.0                      acetoacetamide
3                CHEBI:62885          1.0                   adenosine residue
4                CHEBI:15843          1.0                    arachidonic acid
5                CHEBI:46217          1.0                   L-alanine residue
6                CHEBI:50306          1.0  adenosine 5'-monophosphate residue
7                CHEBI:16449          1.0                             alanine
8                CHEBI:73313          1.0                         Ala-Ala-Ala
9                CHEBI:72816          1.0                  L-alanyl-L-alanine
10               CHEBI:16708          1.0                             adenine
11               CHEBI:16977          1.0                       

In [34]:
print(recom.getSpeciesRecommendation(pred_str='alanine', get_df=True))

                        annotation  match score              label
alanine (cred. 0.943)                                             
1                      CHEBI:16449          1.0            alanine
2                      CHEBI:46217          1.0  L-alanine residue
3                      CHEBI:16977          1.0          L-alanine


In [117]:
df.loc[[1], ['annotation', 'match_score']]

,annotation,match_score
ID: SAM,,
1,CHEBI:59789,1.0


In [62]:
one_dict = {'SAM': df}
one_dict['SAM']

,annotation,match_score,url,label
ID: SAM,,,,
1,CHEBI:59789,1.0,https://www.ebi.ac.uk/chebi/searchId.do?chebiI...,S-adenosyl-L-methionine zwitterion
2,CHEBI:15414,1.0,https://www.ebi.ac.uk/chebi/searchId.do?chebiI...,S-adenosyl-L-methionine


In [146]:
dict.fromkeys(['speies', 'reaction'])

{'speies': None, 'reaction': None}

In [150]:
df.index.name = 'R_PFK (credibility: 0.987)'
df

,annotation,match_score,label,url
R_PFK (credibility: 0.987),,,,
1,RHEA:12420,0.6,tagatose-6-phosphate kinase activity,https://www.rhea-db.org/rhea/12420
2,RHEA:13377,0.6,phosphoglucokinase activity,https://www.rhea-db.org/rhea/13377
